In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.stats import pearsonr, spearmanr
import h3pandas
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle
from warnings import filterwarnings
import matplotlib.colors as colors
import geopandas as gpd 
import sys
import os
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

import matplotlib

from plotting_utils import get_color_dict
color_dict = get_color_dict()

filterwarnings('ignore')
tqdm.pandas()

plt.rcParams.update({'font.family':'arial'})



In [2]:
all_data = pd.read_csv('../../data/D07.BYM2_summary/beta_summary.csv')
all_data = all_data[all_data['niche'].isin(list(get_color_dict().keys()))]

## Beta at mean latitude

In [55]:
from scipy.stats import ttest_rel
p_matrix = np.full(shape=(5,5), fill_value=np.nan)
left_larger_than_right = np.full(shape=(5,5), fill_value=np.nan)
all_data = all_data[all_data['niche']!='all']
for env_var1_count, env_var1 in enumerate(['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin']):
    for env_var2_count, env_var2 in enumerate(['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin']):
        if env_var1_count <= env_var2_count:
            continue
        res = ttest_rel(all_data[all_data['env_var']==env_var1]['mu_beta_mean'].values,
                  all_data[all_data['env_var']==env_var2]['mu_beta_mean'].values)
        p_matrix[env_var1_count, env_var2_count] = res.pvalue
        left_larger_than_right[env_var1_count, env_var2_count] = res.statistic

res = pd.DataFrame(np.sign(left_larger_than_right), columns=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'],
             index=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'])
res_p_matrix = pd.DataFrame(p_matrix, columns=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'],
             index=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'])
for index in list(res.index):
    for col in list(res.columns):
        if np.isnan(res.loc[index, col]):
            continue
        else:
            if res.loc[index, col] > 0:
                sign_ = '>'
            else:
                sign_ = '<'
            
            if res_p_matrix.loc[index, col] < 0.001:
                # pp = 'P < 0.001'
                pp = '***'
            elif res_p_matrix.loc[index, col] < 0.01:
                pp = '**'
            elif res_p_matrix.loc[index, col] < 0.05:
                pp = '*'
            else:
                pp = 'ns.'
            
            new_string = sign_ + ', ' + pp
            
            res.loc[index, col] = new_string
            
            
            print(res.loc[index, col])
    
res

>, ***
>, ***
>, ***
>, ***
>, ***
<, ***
>, ***
>, ***
>, **
>, ***


,mean_NDVI,delta_NDVI,tmax,tmean,tmin
mean_NDVI,NaN,NaN,NaN,NaN,NaN
delta_NDVI,">, ***",NaN,NaN,NaN,NaN
tmax,">, ***",">, ***",NaN,NaN,NaN
tmean,">, ***",">, ***","<, ***",NaN,NaN
tmin,">, ***",">, ***",">, **",">, ***",NaN


## Beta for latitudinal gradient

In [57]:
from scipy.stats import ttest_rel
p_matrix = np.full(shape=(5,5), fill_value=np.nan)
left_larger_than_right = np.full(shape=(5,5), fill_value=np.nan)
all_data = all_data[all_data['niche']!='all']
for env_var1_count, env_var1 in enumerate(['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin']):
    for env_var2_count, env_var2 in enumerate(['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin']):
        if env_var1_count <= env_var2_count:
            continue
        res = ttest_rel(all_data[all_data['env_var']==env_var1]['beta_lat_mean'].values,
                  all_data[all_data['env_var']==env_var2]['beta_lat_mean'].values)
        p_matrix[env_var1_count, env_var2_count] = res.pvalue
        left_larger_than_right[env_var1_count, env_var2_count] = res.statistic

res = pd.DataFrame(np.sign(left_larger_than_right), columns=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'],
             index=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'])
res_p_matrix = pd.DataFrame(p_matrix, columns=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'],
             index=['mean_NDVI', 'delta_NDVI', 'tmax', 'tmean', 'tmin'])
for index in list(res.index):
    for col in list(res.columns):
        if np.isnan(res.loc[index, col]):
            continue
        else:
            if res.loc[index, col] > 0:
                sign_ = '>'
            else:
                sign_ = '<'
            
            if res_p_matrix.loc[index, col] < 0.001:
                # pp = 'P < 0.001'
                pp = '***'
            elif res_p_matrix.loc[index, col] < 0.01:
                pp = '**'
            elif res_p_matrix.loc[index, col] < 0.05:
                pp = '*'
            else:
                pp = 'ns.'
            
            new_string = sign_ + ', ' + pp
            
            res.loc[index, col] = new_string
            
            
            print(res.loc[index, col])
    
res

<, *
<, ***
<, ***
<, ***
<, ***
>, ***
<, ***
<, ***
>, ***
>, **


,mean_NDVI,delta_NDVI,tmax,tmean,tmin
mean_NDVI,NaN,NaN,NaN,NaN,NaN
delta_NDVI,"<, *",NaN,NaN,NaN,NaN
tmax,"<, ***","<, ***",NaN,NaN,NaN
tmean,"<, ***","<, ***",">, ***",NaN,NaN
tmin,"<, ***","<, ***",">, ***",">, **",NaN
